In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [29]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [34]:
union = pd.merge(
    df_his,
    df_track,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

In [35]:
union

,codigo_reserva_atencion,id_paciente,fecha_nacimiento,sexo_x,fecha_reserva,fecha_atencion,id_profesional,nombre_especialidad,codigo_diagnostico,nombre_diagnostico,...,comuna,provincia,region,prevision,plan,tipoatencion,fechacita,horacita,estadocita,hora_completa_cita
0,520078,a1833d4b308a7d80c6d68e7b76d116cdded5fc4867f1ee...,26-09-1999,f,2016-01-12 14:30:00,2016-01-12 14:36:55,d36af679c9dd937adec016b8820e6939987c3fb3a6549f...,CARDIOPATIA CONGENITA,I99X,Otros trastornos y los no especificados del si...,...,Retiro,Linares,Región del Maule,Grupo A,FONASA,Consulta Repetida,12/01/2016,14:30,Atendido,2016-01-12 14:30:00
1,520103,f3e5ec5d5028e79c683926415ecd0bdee75352bf7ce6cf...,06-05-2000,f,2016-03-01 14:30:00,2016-03-01 15:31:06,d36af679c9dd937adec016b8820e6939987c3fb3a6549f...,CARDIOPATIA CONGENITA,I99X,Otros trastornos y los no especificados del si...,...,Punta Arenas,Magallanes,Región de Magallanes y la Antártica Chilena,Grupo C,FONASA,Consulta Repetida,01/03/2016,14:30,Atendido,2016-03-01 14:30:00
2,522345,0745d76bf1d1f1bd2e448da2d2b68fe67bf4946e6bcf21...,02-06-1998,m,2016-03-01 14:00:00,2016-03-01 16:24:44,f9a89a410bf76eceeb958d9b9dfeb5ae30e512bf7f7fd6...,CARDIOPATIA CONGENITA,I99X,Otros trastornos y los no especificados del si...,...,Peñalolén,Santiago,Región Metropolitana,Grupo B,FONASA,Consulta Repetida,01/03/2016,14:00,Atendido,2016-03-01 14:00:00
3,532654,d6a072da3a169ee4b18816044d0e271dd6fcc1caccbcf0...,27-09-1999,m,2016-03-01 14:45:00,2016-03-01 17:40:43,d36af679c9dd937adec016b8820e6939987c3fb3a6549f...,CARDIOPATIA CONGENITA,I99X,Otros trastornos y los no especificados del si...,...,La Serena,Elqui,Región de Coquimbo,Grupo D,FONASA,Consulta Repetida,01/03/2016,14:45,Atendido,2016-03-01 14:45:00
4,536957,c38023a86e94968c62ac139b2bf050b0ab7613174a4d5a...,24-07-1940,m,2016-01-04 14:30:00,2016-01-04 14:50:19,2204e44629c6bc3778a08ccd73fffc852a59fe4cd7b42a...,GES MARCAPASO,I441,Bloqueo auriculoventricular de segundo grado,...,Peñalolén,Santiago,Región Metropolitana,Grupo B,FONASA,Consulta Repetida,04/01/2016,14:30,Atendido,2016-01-04 14:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277155,1000800559,a80b760a414bb9dfc760f9a849776deebbe04c42a9ebd2...,25-11-1965,f,2022-12-30 10:00:00,2022-12-30 10:00:00,ddab3c226078a2580a89d2e7d46503cd33b3069ad1809d...,BRONCOPULMONAR ADULTO INT,J450,Asma predominantemente alérgica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277156,1000800564,9bf72070c65282928e690dd11762cf50ebae014b8678c9...,27-02-1938,m,2022-12-30 11:20:00,2022-12-30 12:44:42,a1048f5c7488844f20e48f6f4c5d309fb99cfc1ce7c25d...,CIRUGIA DE TORAX,J90X,Derrame pleural no clasificado en otra parte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277157,1000800585,fb159e58ef07b2eccc9cbc2e7217554f4a75977a956463...,03-11-1954,m,2022-12-30 11:15:00,2022-12-30 11:15:00,bbaea06982e2048b7b03f34a94c24fe413be834ac8c8e3...,CARDIOLOGIA,XXXX,Otro:,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277158,1000800585,fb159e58ef07b2eccc9cbc2e7217554f4a75977a956463...,03-11-1954,m,2022-12-30 11:15:00,2022-12-30 11:15:00,bbaea06982e2048b7b03f34a94c24fe413be834ac8c8e3...,CARDIOLOGIA,I509,"Insuficiencia cardíaca, no especificada",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
